In [1]:
import pandas as pd
import numpy as np
import camelot
import warnings
warnings.filterwarnings('ignore')
import re

In [2]:
tables = camelot.read_pdf("arizona1.pdf", pages = "all")

In [12]:
table1 = tables[1].df
table1 = table1.iloc[:, 3:11]
table1.loc[table1.index, 8] = table1.loc[table1.index, 8].replace('', '-')

In [13]:
def clean_table(table):
    # create an empty df to populate and return
    cleaned_df = pd.DataFrame(columns = ["Sampling Location", "Sampling Date", "Sampling Time", "Analyte",
                                         "Sampling Method", "Qual", "Amount"])
    
    # iterate through each row in table
    for idx in np.arange(table.shape[0]):
        row_values = np.array(table.iloc[idx].values)
        row_values = row_values[row_values != '']
        
        if len(row_values) < 7:
            # Define a regular expression pattern
            pattern = r'(\d{1,2}:\d{1,2})\s(.*)'

            # Use re.match() to search for the pattern in the text
            match = re.match(pattern, row_values[2])
            time = match.group(1)
            pfas = match.group(2)
            
            row_values = np.concatenate((row_values[:2], [time], [pfas], row_values[-3:]))
        
        cleaned_df.loc[idx] = row_values
        
        cleaned_df["Analyte"] = cleaned_df["Analyte"].replace("\n", " ", regex = True)
    return cleaned_df

In [14]:
arizona1 = clean_table(table1)

In [15]:
for idx in np.arange(2, 5):
    table = clean_table(tables[idx].df)
    arizona1 = pd.concat([arizona1, table], ignore_index = True)

In [16]:
arizona1["Sampling Location ID"] = (["AZ0407093"] * arizona1.shape[0])

In [17]:
arizona1.to_csv("arizona1.csv")